In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print('gpu', gpu)
    tf.config.experimental.set_memory_growth(gpu, True)
    print('memory growth:' , tf.config.experimental.get_memory_growth(gpu))

In [2]:
import pandas as pd
import numpy as np

train_data = pd.read_csv("/home/dawidkubicki/Datasets/fashion-MNIST/train.csv")
test_data = pd.read_csv("/home/dawidkubicki/Datasets/fashion-MNIST/test.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/home/dawidkubicki/Datasets/fashion-MNIST/train.csv'

### Preprocess the data

In [ ]:
mnist_labels = train_data["label"]
mnist_images = train_data.drop(columns=["label"])

In [ ]:
x_train = mnist_images.to_numpy().reshape((60000, 28, 28))
y_train = mnist_labels.to_numpy()
x_test = test_data.to_numpy()

x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=0)

In [6]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(128)
valid_ds = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(128) #in valid we can increase a batch size

test_ds = tf.data.Dataset.from_tensor_slices((x_test)).batch(128)

### Model

In [7]:
class FashNet(tf.keras.Model):
    def __init__(self):
        super(FashNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(16, 3, activation='relu')
        self.maxp1 = tf.keras.layers.MaxPooling2D()
        self.conv2 = tf.keras.layers.Conv2D(32, 3, activation='relu')
        self.maxp2 = tf.keras.layers.MaxPooling2D()
        self.flatten = tf.keras.layers.Flatten()
        self.d1 = tf.keras.layers.Dense(256, activation='relu')
        self.d2 = tf.keras.layers.Dense(10, activation='sigmoid')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxp1(x)
        x = self.conv2(x)
        x = self.maxp2(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
model = FashNet()

In [8]:
#loss and optimizer

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.RMSprop()

In [9]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

valid_loss = tf.keras.metrics.Mean(name='valid_loss')
valid_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='valid_accuracy')

In [10]:
#train step
def train_step(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images, training=True)
        loss = loss_object(labels, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, pred)

In [11]:
def valid_step(images, labels):
    pred = model(images, training=False)
    loss = loss_object(labels, pred)
    
    valid_loss(loss)
    valid_accuracy(labels, pred)

In [ ]:
#training

EPOCHS = 10 

with tf.device('/device:GPU:0'):

    for epoch in range(EPOCHS):
        train_loss.reset_states()
        train_accuracy.reset_states()

        for images, labels in train_ds:
            train_step(images, labels)

        for images, labels in valid_ds:
            valid_step(images, labels)

        print(
            f'Epoch {epoch+1},'
            f'Loss {train_loss.result()}, '
            f'Accuracy {train_accuracy.result() * 100}, '
            f'Valid Loss: {valid_loss.result()},'
            f'Valid Accuracy: {valid_accuracy.result() *100}'
        )

Epoch 1,Loss 0.5980353951454163, Accuracy 78.17500305175781, Valid Loss: 0.42636680603027344,Valid Accuracy: 85.28333282470703
Epoch 2,Loss 0.39199647307395935, Accuracy 85.94583129882812, Valid Loss: 0.3921319246292114,Valid Accuracy: 86.34166717529297
Epoch 3,Loss 0.3353774845600128, Accuracy 87.85832977294922, Valid Loss: 0.36997175216674805,Valid Accuracy: 87.00833129882812
Epoch 4,Loss 0.2998488247394562, Accuracy 89.14374542236328, Valid Loss: 0.3542502820491791,Valid Accuracy: 87.51042175292969
Epoch 5,Loss 0.27431976795196533, Accuracy 90.02291870117188, Valid Loss: 0.34224721789360046,Valid Accuracy: 87.90333557128906
Epoch 6,Loss 0.25265488028526306, Accuracy 90.81041717529297, Valid Loss: 0.3324359357357025,Valid Accuracy: 88.21527862548828
Epoch 7,Loss 0.2338699996471405, Accuracy 91.5, Valid Loss: 0.3248130679130554,Valid Accuracy: 88.45357513427734
Epoch 8,Loss 0.21688660979270935, Accuracy 92.11458587646484, Valid Loss: 0.31849274039268494,Valid Accuracy: 88.651046752929